In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

In [51]:
spark = SparkSession.builder \
    .appName("ETL") \
    .getOrCreate()

In [52]:
data1 = spark.read.csv('../../data/raw/data2.csv',header=True, inferSchema=True)
data2= spark.read.csv('../../data/raw/data3.csv',header=True,inferSchema=True)

In [53]:
## flatten and grouping the first data
column_df=data1.select('skills')
rdd = column_df.rdd.map(lambda row: row[0])
splitted_rdd = rdd.flatMap(lambda line : line.split('·'))
word_pairs1 = splitted_rdd.map(lambda word : (word,1))


In [54]:
df2= data2.groupBy("Skill").agg(sum("Count").alias("Total_count"))
word_pairs2 = df2.rdd.map(lambda row: f"({row['Skill']}: {row['Total_count']})")


In [61]:
union_rdd= word_pairs1.union(word_pairs2)
print(union_rdd.take(50))

[('Experienced ', 1), (' 1 - 3 Yrs of Exp ', 1), (' IT/Software Development ', 1), (' Computer Science ', 1), (' Software Development ', 1), (' Python', 1), ('Entry Level ', 1), (' 0 - 5 Yrs of Exp ', 1), (' IT/Software Development ', 1), (' Education/Teaching ', 1), (' Training/Instructor ', 1), (' Computer Science ', 1), (' Information Technology (IT) ', 1), (' Python ', 1), (' Instructor', 1), ('Experienced ', 1), (' 5 - 8 Yrs of Exp ', 1), (' IT/Software Development ', 1), (' Engineering - Telecom/Technology ', 1), (' React ', 1), (' Django ', 1), (' Full Stack Development ', 1), (' Information Technology (IT) ', 1), (' Computer Science ', 1), (' RESTful API ', 1), (' Microservice Architecture ', 1), (' Database Systems', 1), ('Entry Level ', 1), (' 2+ Yrs of Exp ', 1), (' IT/Software Development ', 1), (' Engineering - Telecom/Technology ', 1), (' Python ', 1), (' Django ', 1), (' Flask ', 1), (' Python Scripting ', 1), (' Web Scraping ', 1), (' WWeb Scraping ', 1), (' CI ', 1), (

In [64]:
trimmed_rdd = union_rdd.map(lambda x: (x[0].strip(), x[1]))

word_count= trimmed_rdd.reduceByKey(lambda a ,b : a+b)
word_count.coalesce(1).saveAsTextFile('../../data/transformed/data_final')